In [1]:
import numpy as np
import gym
import torch.nn as nn
import torch
from collections import deque
import json
import os
import pygame
import math
import random
import matplotlib.pyplot as plt
from gym.utils.play import play

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/cart_pole_dqn')

2023-07-12 11:51:43.634448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 11:51:45.049747: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sugarman/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-07-12 11:51:45.049889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sugarman/.local/lib/python3.10/site-packages/cv2/../../l

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class VfApproxModel(nn.Module):
    """
    Neural Network for Value Function Approximation\n
    Contains Three layers 4->30->30->2
    """
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=4,out_features=120)
        self.layer_2 = nn.Linear(in_features=120,out_features=120)
        self.layer_3 = nn.Linear(in_features=120,out_features=2) # 2 actions as output
        self.relu = nn.ReLU()
    
    def forward(self,features):
        out = self.relu(self.layer_1(features))
        out = self.relu(self.layer_2(out))
        out = self.relu(self.layer_3(out))
        # using softmax as action-selection policy
        out = torch.softmax(out,-1)
        return out

In [5]:
target_policy = VfApproxModel().to(device)
learning_policy = VfApproxModel().to(device)

In [6]:
# Loading weights of learning policy
target_policy.load_state_dict(learning_policy.state_dict())

<All keys matched successfully>

In [7]:
def update_target_policy(TAU):
        """
        Soft update of the target network's weights\n
        θ′ ← τ θ + (1 −τ )θ
        """
        target_net_state_dict = target_policy.state_dict()
        learning_state_dict = learning_policy.state_dict()
        for key in learning_state_dict:
                target_net_state_dict[key] = learning_state_dict[key]*TAU + target_net_state_dict[key] *(1-TAU)
        
        target_policy.load_state_dict(target_net_state_dict)

In [8]:
EPISODES = 10*100
GAMMA = 0.90 # discount factor

ALPHA = 0.001 # Learning rate
EPSILON = 1 # e
EPSILON_DECAY = 0.001
MIN_EXP_RATE = 0.15
MAX_EXP_RATE = 1

REPLAY_LENGTH = 10000
REPLAY_BATCH = 120
TAU = 0.005

In [9]:
criterion = nn.SmoothL1Loss().to(device)
opt = torch.optim.SGD(learning_policy.parameters(), lr=ALPHA)

In [10]:
def batch_state(replay_memory,batch_size):
    """
    Creates Batch of data\n
    (new_state[0],reward[1],running[2],state[3],action[4])
    """
    batch = random.sample(replay_memory,batch_size)
    new_states = []
    states = []
    rewards = []
    done = []
    actions = []
    
    for value in batch:
        new_states.append(value[0])
        rewards.append(value[1])
        done.append(value[2])
        states.append(value[3])
        actions.append(value[4])

    new_states = tuple(new_states)
    rewards = tuple(rewards)
    done = tuple(done)
    states = tuple(states)
    actions = tuple(actions)
    
    new_states = torch.cat(new_states)
    new_states = new_states.reshape(-1,4)

    states = torch.cat(states)
    states = states.reshape(-1,4)

    rewards = torch.cat(rewards)
    rewards = rewards.reshape(-1,1)

    done = torch.cat(done)
    done = done.reshape(-1,1)

    actions = torch.cat(actions)
    actions = actions.reshape(-1,1)
    

    return new_states,rewards,done,states,actions


In [11]:
def q_update(state,new_state,reward,done,actions):
    
    # current q values by action
    state_action_value = learning_policy(state).gather(1,actions)
    # max values (expected)
    with torch.no_grad():
        next_state_values = learning_policy(new_state).max(1)[0]
        next_state_values = next_state_values.reshape(-1,1)
    
    expected_state_action_values = (next_state_values * GAMMA) * done + reward
    
    loss = criterion(state_action_value,expected_state_action_values)
    
    opt.zero_grad()
    loss.backward()
    # torch.nn.utils.clip_grad_value_(learning_policy.parameters(), 100)
    opt.step()

    return loss,expected_state_action_values,state_action_value

In [12]:
# new_states,rewards,done,states,actions = batch_state(mem,REPLAY_BATCH)
# loss,exp_s_a,s_a_v = q_update(states,new_states,rewards,done,actions)

In [13]:
# exp_s_a,s_a_v

In [12]:
class ReplayMemory:
    """
        Replay Memory for string Experience
    """
    def __init__(self,length,batch_size):
        self.replay_memory = deque(maxlen=length)
        self.batch_size = batch_size
    
    def add_experience(self,new_state,reward,running,state,action):
        """
            Adds Experience into replay_memory\n
            new_state and state both are torch tensors
        """
        self.replay_memory.append((new_state,reward,running,state,action))
    
    def sample(self,batch_size):
        pass
        
    
    def train_on_replay(self):
        """
            Training on Replay memory
        """
        batch = random.sample(self.replay_memory,self.batch_size)
        
        for new_state,reward,running,state,action in batch:

            loss = q_update(state,new_state,reward,running)
        
        return loss # final loss of replay batch


In [13]:
replay_memory = ReplayMemory(REPLAY_LENGTH,batch_size=REPLAY_BATCH)

In [14]:
state = torch.tensor([[0,0,4,0],[0,0,1,2],[1,2,3,4]],dtype=torch.float32,device=device)
state

tensor([[0., 0., 4., 0.],
        [0., 0., 1., 2.],
        [1., 2., 3., 4.]], device='cuda:0')

In [15]:
steps_done = 0
def select_action(state,env):
    global steps_done
    sample = random.random()
    eps_threshold = MIN_EXP_RATE + (EPSILON - MIN_EXP_RATE) * \
        math.exp(-1. * steps_done / EPSILON_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return learning_policy(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


In [31]:
env = gym.make("CartPole-v1")
for episode in range(EPISODES):
    done = False
    state,info = env.reset()
    reward_per_episode = 0
    old_reward = 0
    state = torch.tensor(state,dtype=torch.float32,requires_grad=True,device=device).unsqueeze(0)
    print(f"-------Episode:[{episode+1}/{EPISODES}]--------")
    step = 1
    while not done:
        action = learning_policy(state).max(1)[1].view(1, 1)
        
        new_state,reward,terminated,truncated,info = env.step(action.item())
        
        # if step % 10 == 0:
        #     reward += 1
        new_state = torch.tensor(new_state,dtype=torch.float32,requires_grad=True,device=device).unsqueeze(0)
        
        done = terminated or truncated
        # if done:
        #     new_state = torch.tensor([0,0,0,0],dtype=torch.float32,requires_grad=True,device=device).unsqueeze(0)
        running_data = torch.tensor([int(not done)],dtype=torch.int16,device=device)

        reward = torch.tensor([reward],dtype=torch.int64,device=device)
        action = torch.tensor([action],dtype=torch.int64,device=device)

        replay_memory.add_experience(new_state,reward,running_data,state,action)
        mem = replay_memory.replay_memory
        
        reward_per_episode += reward.item()
        state = new_state
        step +=1
        done = terminated or truncated
        
    if len(mem) > REPLAY_BATCH:
        # print("Training")
        new_states,rewards,done,states,actions = batch_state(mem,REPLAY_BATCH)
        loss,exp,sa = q_update(states,new_states,rewards,done,actions)
        
    # saving weights with high reward
    # print(reward_per_episode,old_reward)
    if reward_per_episode > old_reward:
        print("Saving Weights")
        torch.save(learning_policy.state_dict(),f"weights/weights_{reward_per_episode}.pth")
    old_reward = reward_per_episode
    # q_updating after batch sampling

    print("Reward per Episode: ",reward_per_episode)
    writer.add_scalar("Exploration Rate",EPSILON,episode)
    EPSILON = (MAX_EXP_RATE-MIN_EXP_RATE) * np.exp(-EPSILON_DECAY*episode) + MIN_EXP_RATE
    writer.add_scalar("Reward per Episode",reward_per_episode,episode)
    if len(mem) > REPLAY_BATCH:
        writer.add_scalar("Loss",loss.item(),episode)
        print("Loss per Episode",loss.item())
    
    
    if episode % 30 == 0:
        print("[Updating Policy]")
        update_target_policy(TAU)

env.close()    

-------Episode:[1/1000]--------
Saving Weights
Reward per Episode:  48
Loss per Episode 0.5002292394638062
[Updating Policy]
-------Episode:[2/1000]--------
Saving Weights
Reward per Episode:  40
Loss per Episode 0.4971349537372589
-------Episode:[3/1000]--------
Saving Weights
Reward per Episode:  33
Loss per Episode 0.4975564479827881
-------Episode:[4/1000]--------
Saving Weights
Reward per Episode:  48
Loss per Episode 0.504112720489502
-------Episode:[5/1000]--------
Saving Weights
Reward per Episode:  60
Loss per Episode 0.4912559986114502
-------Episode:[6/1000]--------
Saving Weights
Reward per Episode:  33
Loss per Episode 0.5103219151496887
-------Episode:[7/1000]--------
Saving Weights
Reward per Episode:  46
Loss per Episode 0.5090181827545166
-------Episode:[8/1000]--------
Saving Weights
Reward per Episode:  52
Loss per Episode 0.5291661620140076
-------Episode:[9/1000]--------
Saving Weights
Reward per Episode:  34
Loss per Episode 0.4956069886684418
-------Episode:[10/1

In [32]:
torch.save(learning_policy.state_dict(),"weights_test.pth")

In [20]:
# exp_s_a,s_a_v

In [33]:
learning_policy.load_state_dict(torch.load("weights/weights_185.pth"))

<All keys matched successfully>

In [40]:
env = gym.make("CartPole-v1",render_mode="human")
done = False
state,info = env.reset()
reward_per_episode = 0
state = torch.tensor(state,dtype=torch.float32,requires_grad=True,device=device)
while not done:
    action = learning_policy(state).max(0)[1]
        
    new_state,reward,terminated,truncated,info = env.step(action.item())
    new_state = torch.tensor(new_state,dtype=torch.float32,requires_grad=True,device=device)
    done = terminated or truncated
    if done:
        print(new_state)
    reward_per_episode+=1
    state = new_state

print(reward_per_episode)
env.close()

tensor([0.0162, 0.5133, 0.2183, 0.3421], device='cuda:0', requires_grad=True)
73


In [23]:
# exp_s_a,s_a_v

In [24]:
# state_action_value = learning_policy(state).max(0)[0]
# state_action_value 

In [25]:
# mem = replay_memory.replay_memory
# new_states,rewards,done,states,actions = batch_state(mem,120)
# print(new_states.shape)
# print(rewards.shape)
# print(done.shape)
# print(states.shape)
# print(actions.shape)

In [26]:
# states = learning_policy(states).gather(1,actions)
# with torch.no_grad():
#     next_state_values = target_policy(new_states).max(1)[0]
# next_state_values = next_state_values.reshape(-1,1)
# print(next_state_values.shape)
# expected_state_action_values = ((next_state_values * GAMMA) + rewards) * done
# print(rewards.shape)
# print(states.shape)
# print(expected_state_action_values.shape)

In [27]:
# loss = criterion(states,expected_state_action_values)
# print(loss)
# opt.zero_grad()
# loss.backward()
# opt.step()
# loss = criterion(states,expected_state_action_values)
# print(loss)

In [28]:
pygame.display.quit() 